## Multiple Sequence Alignment of Spinosyn candidate clusters
This analysis requires project `TIGR03997` containing 17 BGCs that have match with ARTS model TIGR03997. The project was run through the `BGC` comparison subworkflow by running `bgcflow run --snakefile workflow/BGC`

In [ ]:
%%bash
pip install pyarrow
pip install networkx
pip install biopython

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from pathlib import Path
from Bio import SeqIO
import yaml

%load_ext rpy2.ipython

In [ ]:
%%R
library(tidyverse)
library(gggenomes)
library(patchwork)  # arrange multiple plots
library(ggtree) 
library("phangorn")

## File configuration

In [ ]:
with open("config.yaml", "r") as f:
    notebook_configuration = yaml.safe_load(f)
notebook_configuration

In [ ]:
bgcflow_dir = Path(notebook_configuration["bgcflow_dir"])
PROJECT_NAME = "Spinosyn"
PROJECT_NAME2 = "mq_saccharopolyspora"

report_dir = bgcflow_dir / f"data/processed/{PROJECT_NAME2}"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"

SAMPLE_FILE_NAME = "samples.csv"
ANTISMASH_VERSION = "6.1.1"
FIGURE = "Figure_S10"
FIGURE_N = 'b'
FIGURE_MASH = "Figure_3"

In [ ]:
samples_config = pd.read_csv(PROJECT_CONFIG_DIR / SAMPLE_FILE_NAME)
samples_config

### Extract COG information from mmseqs2

In [ ]:
G = nx.read_edgelist(bgcflow_dir / f"data/interim/mmseqs2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}_cluster.tsv")
# get the connected components of G
components = nx.connected_components(G)

# sort the components based on their size
sorted_components = sorted(components, key=lambda x: len(x), reverse=True)

result = {"feature_id": [], "cluster_id" : [], "cluster_n" : [], "locus_tag" : []}
for num, g in enumerate(sorted_components):
    size = len(g)
    for item in g:
        result['feature_id'].append(f"cds-{item}")
        result['cluster_id'].append(f"cog_{num+1:02d}")
        result['cluster_n'].append(size)
        result['locus_tag'].append(item)
df_mmseqs2 = pd.DataFrame.from_dict(result)
df_mmseqs2.to_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_mmseqs_cog.csv", index=False)

### Preparation of the dataset using minimap2 and mmseqs
This analysis refers to the example shown here: https://thackl.github.io/gggenomes/articles/emales.html

Here, COGs were defined using mmseqs2 and links between genes are create using minimap2

In [ ]:
region_seq_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.fna")
region_feat_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.gbk")
region_minimap2_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.paf")
cogs_raw = f"assets/tables/{FIGURE}_{PROJECT_NAME}_mmseqs_cog.csv"

In [ ]:
%%R -i region_seq_raw,region_feat_raw,region_minimap2_raw,cogs_raw
region_seq <- read_seqs(region_seq_raw)
region_feat <- read_feats(region_feat_raw)
region_link <- read_paf(region_minimap2_raw)
cogs <- read_csv(cogs_raw)

cogs %<>% mutate(
  cluster_label = paste0(cluster_id, " (", cluster_n, ")"),
  cluster_label = fct_lump_min(cluster_label, 1, other_level = "rare"),
  cluster_label = fct_lump_min(cluster_label, 5, other_level = "medium"),
  cluster_label = fct_relevel(cluster_label, "rare", after=Inf))

merged_feat <- merge(region_feat, cogs, by.x = "feat_id", by.y = "feature_id")

In [ ]:
ggenomes_feat_table = f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv"

In [ ]:
%%R -i ggenomes_feat_table
write_csv(merged_feat, ggenomes_feat_table)

### GG Genomes Vis
This is the default GG Genomes visualization using mmseqs2 as links, limited to the first 25000 bp

In [ ]:
%%R # Manipulate length of the gene clusters
#region_seq['length'][region_seq['seq_id'] == merged_feat[353+1,'seq_id']] <- merged_feat[353+1,'end']
#region_seq['length'][region_seq['seq_id'] == merged_feat[86,'seq_id']] <- merged_feat[86,'end']
#region_seq['length'][region_seq['seq_id'] == 'NZ_CP061007.1.region011'] <- 25000
#region_seq['length'][region_seq['seq_id'] == 'NZ_GL877879.1.region006'] <- 25000
#region_seq['length'][region_seq['seq_id'] == 'NZ_JACHIW010000001.1.region012'] <- 25000
#region_seq['length'][region_seq['seq_id'] == 'NZ_CP040605.1.region009'] <- 25000
#region_seq['length'] <- 25000
region_seq

In [ ]:
%%R -w 1200 -h 600
p <- gggenomes(
    genes=merged_feat,  # a gene track, added as first feat track
    seqs=region_seq) %>% add_links(region_link) %>% flip_seqs(2,4,5,6)

p2_mmseqs2 <- p +
    geom_seq(aes(color=strand), arrow=TRUE) + 
    geom_seq() + geom_bin_label() +
    geom_link(offset = c(0.2, 0.2), color="white", alpha=.8) +  # the first link track
    geom_gene(position = "identity", aes(fill=cluster_label)) +  # the first feat track filtered for geneish feats: CDS, mRNA, ..
    labs(caption="Nucleotide Size") #+ 
    #scale_fill_manual("Conserved genes", 
    #                  labels=sort(unique(merged_feat$cluster_label)),
    #                  values=c("#f94144","#f3722c","#f8961e","#f9844a","#f9c74f","#90be6d","#43aa8b","#4d908e","#577590","#277da1", 
    #                           'white', 'white', 'white', 'white', 'white'))
p2_mmseqs2

## Utilize Clinker for Links
Now, we will use clinker instead of mmseqs2 for the links

In [ ]:
with open(bgcflow_dir / f"data/processed/{PROJECT_NAME}/clinker/{ANTISMASH_VERSION}/clinker.txt", "r") as f:
    clinker_data = f.readlines()

clinker_dict = {}
container = {}
for num, line in enumerate(clinker_data):
    if line.startswith("----"):
        header = clinker_data[num - 1].strip("\n")
        if header not in clinker_dict.keys():
            clinker_dict[header] = []
    else:
        if line.startswith("----") or line.startswith("Query") or "vs" in line:
            pass
        else:
            data = line.strip("\n").split()
            seq_id = header.split(" vs ")
            if len(data) == 0:
                data = [None, None, None, None]
            data = seq_id + data
            assert len(data) == 6, data
            clinker_dict[header].append(data)
            container[num] = data
            

df_clinker = pd.DataFrame.from_dict(container, orient="index", columns=["seq_id", "seq_id2", "locus_tag", "locus_tag2", "identity", "similarity"])
df_clinker.identity = df_clinker.identity.astype(float)
df_clinker = df_clinker[df_clinker.identity >= 0.7]
df_clinker

In [ ]:
cds_dict = {}
for i in samples_config.index:
    gbk = bgcflow_dir / samples_config.loc[i, "gbk_path"]
    seq_id = samples_config.loc[i, "bgc_id"]
    print(seq_id)
    with open(gbk, "r") as input_handle:
        for record in SeqIO.parse(input_handle, "genbank"):
            antismash_data = record.annotations['structured_comment']['antiSMASH-Data']
            for feature in record.features:
                if feature.type == "CDS":
                    translation = feature.qualifiers["translation"][0]  
                    strand = feature.location.strand
                    if strand > 0:
                        start, end = int(feature.location.start), int(feature.location.end)
                    elif strand < 0:
                        end, start = int(feature.location.start), int(feature.location.end)
                    try:
                        locus_tag = feature.qualifiers["locus_tag"][0]
                    except KeyError:
                        print(seq_id, feature.qualifiers.keys())
                        locus_tag = feature.qualifiers["protein_id"][0]
                        print(locus_tag)
                    if seq_id.startswith("BGC"):
                        locus_tag = feature.qualifiers["protein_id"][0]
                    cds_dict[locus_tag] = {"seq_id" : seq_id, "start" : start, "end" : end, "strand" : strand, "translation":translation}

In [ ]:
for i in df_clinker.index:
    locus_tag1 = df_clinker.loc[i, "locus_tag"]
    locus_tag2 = df_clinker.loc[i, "locus_tag2"]
    try:
        if locus_tag1 != None:
            df_clinker.loc[i, "start"] = int(cds_dict[locus_tag1]['start'])
            df_clinker.loc[i, "end"] = int(cds_dict[locus_tag1]['end'])
    except KeyError as e:
        print(e, df_clinker.loc[i, "seq_id"])
    try:
        if locus_tag2 != None:
            df_clinker.loc[i, "start2"] = int(cds_dict[locus_tag2]['start'])
            df_clinker.loc[i, "end2"] = int(cds_dict[locus_tag2]['end'])
    except KeyError as e:
        print(e, df_clinker.loc[i, "seq_id2"])


In [ ]:
region_clinker_raw = f"assets/tables/{FIGURE}_{PROJECT_NAME}_clinker_links.csv"
df_clinker.dropna().to_csv(region_clinker_raw, index=False)

In [ ]:
%%R -i region_seq_raw,region_feat_raw,region_clinker_raw,cogs_raw
region_seq <- read_seqs(region_seq_raw)
region_feat <- read_feats(region_feat_raw)
region_link <- read_csv(region_clinker_raw)
cogs <- read_csv(cogs_raw)

cogs %<>% mutate(
  cluster_label = paste0(cluster_id, " (", cluster_n, ")"),
  cluster_label = fct_lump_min(cluster_label, 1, other_level = "rare"),
  cluster_label = fct_lump_min(cluster_label, 3, other_level = "medium"),
  cluster_label = fct_relevel(cluster_label, "rare", after=Inf))

merged_feat <- merge(region_feat, cogs, by.x = "feat_id", by.y = "feature_id")

In [ ]:
%%R -w 1200 -h 600
#region_seq['length'] <- 20000

p <- gggenomes(
    genes=merged_feat,  # a gene track, added as first feat track
    seqs=region_seq) %>% add_links(region_link) %>% flip_seqs(2,4,5,6)

p2_clinker <- p + geom_seq(aes(color=strand), arrow=TRUE) + 
    geom_seq() + geom_bin_label() +
    geom_link() +  # the first link track
    geom_gene(position = "identity", aes(fill=cluster_label)) +  # the first feat track filtered for geneish feats: CDS, mRNA, ..
    labs(caption="Nucleotide Size") #+
    #scale_fill_manual("Conserved genes", 
    #                  labels=sort(unique(merged_feat$cluster_label)),
    #                  values=c("#f94144","#f3722c","#f8961e","#f9844a","#f9c74f","#90be6d","#43aa8b","#4d908e","#577590","#277da1", 
    #                           'white', 'white', 'white', 'white'))
p2_clinker

In [ ]:
%%R -w 1800 -h 600
p2_clinker + p2_mmseqs2

### Adding BGC tree based on multiple alignment of selected COGs

In [ ]:
%%bash
mamba install -c bioconda clustalo iqtree seqkit trimal -y

In [ ]:
df = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv")
correct_naming = df.set_index('locus_tag.y').loc[:, ["seq_id", 'cluster_id']].T.to_dict()

In [ ]:
# generate tree
n_bgc = len(df.seq_id.unique())
for cog in df.cluster_label.unique():
    if cog not in ['rare', 'medium']:
        subset = df[df.cluster_label == cog]
        product = subset['product'].unique()
        bgc_dist = subset.seq_id.unique()
        if len(bgc_dist) == n_bgc:
            cog_n = int(cog.split(" ")[1].strip("(").strip(")"))
            print(len(bgc_dist), n_bgc, cog_n)

            print(cog, "\n* ".join(product), "\n* ".join(subset.feat_id))
            print("")
subset

From COG distribution, we can see that only COG_02 match criteria to build MSA

In [ ]:
with open(bgcflow_dir / f"data/interim/mmseqs2/{PROJECT_NAME}/msa_{PROJECT_NAME}_{ANTISMASH_VERSION}.db") as f:
    data = f.readlines()
    for num, i in enumerate(data):
        if i.startswith(">"):
            rename = i.split()
            locus_tag = rename[0].replace(">", "")
            seq_id = correct_naming[locus_tag]['seq_id']
            data[num] = " ".join([f'>{seq_id}', locus_tag, "\n"])

            
number_of_bgcs = len(df.seq_id.unique())
for num, i in enumerate(data):
    if ("#cl" in i):
        count = [int(x.strip("n=")) for x in i.split() if "n=" in x][0]
        if count in [number_of_bgcs]:
            msa = data[(num+2):(num+2)+(count*2)]
            if len(set([seq.split()[0] for seq in msa if seq.startswith(">")])) in [number_of_bgcs]:
                filename = i.split("|")[0].replace("#", "").replace('\0', '')
                filename = Path(f'assets/data/{FIGURE}/msa/{PROJECT_NAME}/{filename}_aligned.fasta')
                filename.parent.mkdir(exist_ok=True, parents=True)
                print(filename, i, count)
                with open(filename, "w") as outfile:
                    outfile.writelines(msa)

In [ ]:
treefile = f"assets/data/{FIGURE}/msa/{PROJECT_NAME}/core_BGC_concat_clean.fasta.treefile"
Path(treefile).parent.mkdir(parents=True, exist_ok=True)

In [ ]:
%%bash -s "$PROJECT_NAME" "$FIGURE" "$treefile" # USE FOR MULTIPLE GENES

if [ -f $3 ]; then
    echo "File $3 already exists. Skipping command."
else
    echo "Building Tree..."
    seqkit concat -o assets/data/$2/msa/$1/core_BGC_concat.fasta assets/data/$2/msa/$1/*_aligned.fasta
    trimal -in assets/data/$2/msa/$1/core_BGC_concat.fasta -out assets/data/$2/msa/$1/core_BGC_concat_clean.fasta
    iqtree -s assets/data/$2/msa/$1/core_BGC_concat_clean.fasta --redo
fi

In [ ]:
# Prepare data tables
gtdtbk = bgcflow_dir / f"data/processed/{PROJECT_NAME}/tables/df_gtdb_meta.csv"
mash = Path(f"assets/tables/{FIGURE_MASH}b_mash_hcluster.csv")
ncbi = report_dir / "tables/df_ncbi_meta.csv"

# merge data tables for tree
df_mash = pd.read_csv(mash, index_col=0)
df_gtdbtk = pd.read_csv(gtdtbk, index_col=0)
df_gtdbtk = df_gtdbtk.loc[:, ["Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species", "Organism"]]
df_tree_annotation = pd.concat([df_mash, df_gtdbtk], axis=1, join="inner")
df_ncbi = pd.read_csv(ncbi, index_col=0).loc[:, "strain"]
df_tree_annotation = pd.concat([df_tree_annotation, df_ncbi], axis=1, join="inner")

## manual annotation for tree
### Rename MASH based Species Phylogroup
# df["Species_Phylogroup"] = [f"P{i + 1}" for i in df.hcluster]
df_tree_annotation = df_tree_annotation#.reset_index(drop=False).rename(columns={"index" : "genome_id"})
tree_mapping = df_tree_annotation.T.to_dict()
df_tree_annotation.columns
df_tree_annotation

In [ ]:
samples_config = pd.read_csv(PROJECT_CONFIG_DIR / SAMPLE_FILE_NAME)
for i in samples_config.index:
    genome_id = samples_config.loc[i, "genome_id"]
    for c in df_tree_annotation.columns:
        if genome_id in tree_mapping.keys():
            samples_config.loc[i, c] = tree_mapping[genome_id][c]

In [ ]:
### Rename MIBIG entries
for i in samples_config.index:
    if samples_config.loc[i, "bgc_id"].startswith("BGC"):
        samples_config.loc[i, "bgc_id"] = samples_config.loc[i, "bgc_id"] + ".region001"
        samples_config.loc[i, "phylogroup"] = "MIBIG"
    elif samples_config.loc[i, "bgc_id"] == "NC_000962.3.region003":
        samples_config.loc[i, "phylogroup"] = "M. tuberculosis"
        samples_config.loc[i, "color_code"] = "red"
### Set up tip labels to show
#samples_config['tip_label'] = [f"{samples_config.loc[i, 'phylogroup']} | {samples_config.loc[i, 'bgc_id']} | {samples_config.loc[i, 'Organism'].replace('s__Saccharopolyspora','S')} ({samples_config.loc[i, 'strain']})" for i in samples_config.index]
samples_config['tip_label'] = [f"{samples_config.loc[i, 'phylogroup']} | {samples_config.loc[i, 'bgc_id']}" for i in samples_config.index]

# save to intermediate file
tree_annotation_file = f"assets/tables/{FIGURE}_{PROJECT_NAME}_ggtree.csv"
samples_config.to_csv(tree_annotation_file, index=False)

In [ ]:
%%R -w 800 -h 600 -i treefile,tree_annotation_file
tree <- read.tree(treefile)
tree_data <- read.csv(tree_annotation_file)

# midpoint root
tree <- phangorn::midpoint(tree)
tree <- ladderize(reorder(tree))

t <- ggtree(tree)
t <- t %<+% tree_data

t2 = t + geom_tiplab(aes(label=tip_label, fill=phylogroup),
                     size=3, hjust=-0.03, family='sans', align=T,
                    linetype = "dotted", linesize = 1) + hexpand(1.5)
    
t3 = t2 + geom_tippoint(size=2.6, alpha=0.8, aes(color=phylogroup, stroke=1)) + 
        scale_color_manual(values = c('P1'='#264653', 'P2'='#e9c46a', 'P3'='#808080', 
                           'P4'='#808080', 'P5'='#f4a261', 'P6'='#808080', 
                           'P7'='#e76f51', 'M. tuberculosis'='red')) + 
        theme(legend.position='none')

fig <- t3 + scale_y_continuous(expand=c(0.01,0.7,0.01,0.7)) + p2_clinker %>% pick_by_tree(t2) + plot_layout(widths = c(2,2))
fig

### Add mapping to known reference using CBlaster

In [ ]:
cblaster_db_path = str(bgcflow_dir / "data/processed/mq_saccharopolyspora/cblaster/bgcs/6.1.1/cblaster_bgc_db.dmnd")
query_gbk = bgcflow_dir / "data/external/bgc_selection/MIBIG/BGC0000148.region001.gbk"
cblaster_out = Path(f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_{query_gbk.stem}.csv")
cblaster_out.parent.mkdir(parents=True, exist_ok=True)
query_gbk = str(query_gbk)
cblaster_out = str(cblaster_out)

In [ ]:
%%bash -s "$cblaster_db_path" "$query_gbk" "$cblaster_out"
# first, create env using mamba 
# mamba create -f <your bgcflow dir>/workflow/envs/cblaster.yaml
mamba run -n cblaster cblaster search -m local -db $1 -qf $2 -o $3

In [ ]:
with open(cblaster_out, "r") as f:
    cblaster = f.readlines()

cluster_marker = []    
for num, line in enumerate(cblaster):
    if line.startswith("Cluster"):
        cluster_marker.append(num)
container = {'Query' : [], 'Subject' : [], 'Identity' : [], 'Coverage' : [], 'E-value' : [], 'Bitscore' : [], 'Start' : [], 'End' : [], 'Strand' : []}
for num, item in enumerate(cluster_marker):
    if num == len(cluster_marker)-1:
        data = [x.split() for x in cblaster[item+2:]]
    else:
        data = [x.split() for x in cblaster[item+2:cluster_marker[num+1]-6]]
    for v in data:
        for n, c in enumerate(container.keys()):
            container[c].append(v[n])
df_feat = pd.read_parquet(report_dir / "data_warehouse/6.1.1/cdss.parquet")
seq_id_mapping = df_feat.set_index("locus_tag").region_id.to_dict()

In [ ]:
df_cblaster_hits = pd.DataFrame.from_dict(container)
df_cblaster_hits.Query.unique()

In [ ]:
cblaster_cds_dict = {}
seq_id = Path(query_gbk).stem
with open(query_gbk, "r") as input_handle:
    for record in SeqIO.parse(input_handle, "genbank"):
        antismash_data = record.annotations['structured_comment']['antiSMASH-Data']
        for feature in record.features:
            if feature.type == "CDS":
                start, end = int(feature.location.start), int(feature.location.end)
                strand = feature.location.strand
                translation = feature.qualifiers["translation"][0]
                try:
                    gene = feature.qualifiers['gene'][0]
                except KeyError:
                    #print(feature.qualifiers)
                    gene = feature.qualifiers['product'][0]
                try:
                    locus_tag = feature.qualifiers["locus_tag"][0]
                    #print(feature.qualifiers)
                except KeyError:
                        #print(seq_id, feature.qualifiers.keys())
                        locus_tag = feature.qualifiers["protein_id"][0]
                        print(locus_tag)
                #if seq_id.startswith("BGC"):
                #        locus_tag = feature.qualifiers["protein_id"][0]
                cblaster_cds_dict[locus_tag] = {"seq_id" : seq_id, "start" : start, "end" : end, 
                                                "strand" : strand, "translation":translation,
                                               "name" : gene}
df_cds_cblaster = pd.DataFrame.from_dict(cblaster_cds_dict).T
df_cds_cblaster

In [ ]:
df_query_mapping = df_cds_cblaster.loc[df_cblaster_hits.Query.unique()]
q_map = df_query_mapping.loc[:, ['name', 'seq_id']].T.to_dict()
#q_map['SACE_4230']['name'] = 'EryKC'
#q_map['SACE_4231']['name'] = 'EryS'
df_cblaster_hits['query_description'] = [q_map[i]['name'] for i in df_cblaster_hits.Query]
df_cblaster_hits['seq_id'] = [seq_id_mapping[i] for i in df_cblaster_hits.Subject]
#df_cblaster_hits['feat_id'] = [f"cds-{i}" for i in df_cblaster_hits.Subject]
df_cblaster_hits

In [ ]:
ctr = len(df_cblaster_hits)
description_map = df_cblaster_hits.set_index("Query").loc[:, "query_description"].to_dict()
for query_self in df_cblaster_hits.Query.unique():
    print(query_self)
    df_cblaster_hits.loc[ctr, "Query"] = query_self
    df_cblaster_hits.loc[ctr, "Subject"] = query_self
    df_cblaster_hits.loc[ctr, "Identity"] = 100
    df_cblaster_hits.loc[ctr, "Coverage"] = 100
    df_cblaster_hits.loc[ctr, "E-value"] = 0
    df_cblaster_hits.loc[ctr, "Bitscore"] = 999
    df_cblaster_hits.loc[ctr, "Start"] = cblaster_cds_dict[query_self]['start']
    df_cblaster_hits.loc[ctr, "End"] = cblaster_cds_dict[query_self]['end'] - 10
    if cblaster_cds_dict[query_self]['strand'] > 0:
        df_cblaster_hits.loc[ctr, "Strand"] = "+"
    else:
        df_cblaster_hits.loc[ctr, "Strand"] = "-"
    df_cblaster_hits.loc[ctr, "seq_id"] = cblaster_cds_dict[query_self]['seq_id']
    df_cblaster_hits.loc[ctr, "query_description"] = description_map[query_self]
    ctr = ctr + 1 


In [ ]:
df_cog = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_mmseqs_cog.csv")
df_cog = df_cog.merge(df_cblaster_hits, left_on="locus_tag", right_on="Subject", how="outer")
df_cog = df_cog.rename(columns={'feature_id':'feat_id', 'Start':'start', 'End':'end', 'region_id':'seq_id', 'Query':'blast_hit', 'query_description':'blast_desc'})  
df_cog = df_cog.dropna().loc[:, ['feat_id', 'start', 'end', 'seq_id', 'blast_hit', 'blast_desc', 'cluster_id']]

In [ ]:
df_cog['end'] = df_cog['end'].astype(int) - df_cog['start'].astype(int)
df_cog['start'] = 0
blast_hits = f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_blast_hits.csv"
df_cog.to_csv(blast_hits, index=False)

In [ ]:
%%R -i blast_hits
blast <- read_csv(blast_hits)
blast

In [ ]:
region_seq_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.fna")
region_feat_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.gbk")
region_minimap2_raw = str(bgcflow_dir / f"data/interim/minimap2/{PROJECT_NAME}/{PROJECT_NAME}_{ANTISMASH_VERSION}.paf")
cogs_raw = f"assets/tables/{FIGURE}_{PROJECT_NAME}_mmseqs_cog.csv"

In [ ]:
%%R -i region_seq_raw,region_feat_raw,region_minimap2_raw,cogs_raw
region_seq <- read_seqs(region_seq_raw)
region_feat <- read_feats(region_feat_raw)
region_link <- read_paf(region_minimap2_raw)
region_link <- read_csv(region_clinker_raw)
cogs <- read_csv(cogs_raw)

cogs %<>% mutate(
  cluster_label = paste0(cluster_id, " (", cluster_n, ")"),
  cluster_label = fct_lump_min(cluster_label, 2, other_level = "rare"),
  cluster_label = fct_lump_min(cluster_label, 6, other_level = "medium"),
  cluster_label = fct_relevel(cluster_label, "rare", after=Inf))

merged_feat <- merge(region_feat, cogs, by.x = "feat_id", by.y = "feature_id")

In [ ]:
%%R -w 1200 -h 600
#region_seq['length'] <- 30000

p <- gggenomes(
    genes=merged_feat,  # a gene track, added as first feat track
    seqs=region_seq) %>% add_links(region_link) %>% add_subfeats(blast, .transform = "none") %>% flip_seqs(2,4,5,6)

p2_clinker <- p + geom_seq(aes(color=strand), arrow=TRUE) + 
    geom_seq() + #geom_bin_label() +
    geom_link() +  # the first link track
    geom_gene(position = "identity", aes(fill=cluster_label)) +  # the first feat track filtered for geneish feats: CDS, mRNA, ..
    labs(caption="Nucleotide Size") +
    scale_fill_manual("Conserved genes", 
                      labels=sort(unique(merged_feat$cluster_label)),
                      values=c("#f94144","#f3722c","#f8961e","#f9844a","#f9c74f","#90be6d","#43aa8b","#4d908e","#577590","#277da1", 
                               'white', 'white', 'white', 'white','white', 'white', 'white', 'white', 'white'))
p2_clinker

In [ ]:
outfile_svg = f"assets/figures/{FIGURE}/{FIGURE}b.svg"
outfile_pdf = f"assets/figures/{FIGURE}/{FIGURE}b.pdf"
Path(outfile_svg).parent.mkdir(exist_ok=True, parents=True)

In [ ]:
%%R -w 800 -h 600 -i outfile_svg,outfile_pdf
library("svglite")

p <- gggenomes(
    genes=merged_feat,  # a gene track, added as first feat track
    seqs=region_seq) %>% add_links(region_link) %>% add_subfeats(blast, .transform = "none") %>% flip_seqs(2,4,5,6)


p2 <- p +
    geom_seq(aes(color=strand), arrow=TRUE) + 
    geom_seq() + #geom_bin_label(size = 3.5, expand_left =.7) + 
    geom_link(offset = c(0.2, 0.2), color="white", alpha=.8) +  # the first link track
    geom_gene(position = "identity", aes(fill=cluster_label)) +  # the first feat track filtered for geneish feats: CDS, mRNA, ..
    labs(caption="Size (bp)") + 
    geom_feat(aes(color=blast_desc), size=1, position="strandpile") +
    #scale_fill_manual("Conserved genes", 
    #                  labels=sort(unique(merged_feat$cluster_label)),
    #                  values=c(
    #                           "#f94144","#f94144","#f94144","#f94144","#f3722c","#f8961e","#f9844a","#f9c74f", 
    #                           "#90be6d","#43aa8b","#4d908e","#577590","#277da1", 
    #                           'grey', 'grey', 'white', 'white', 'white')) +
    scale_color_viridis_d("Blast hits & Features", direction = -1) +
    scale_linetype("Graphs") +  theme(legend.position="none")

fig <- t3 + scale_y_continuous(expand=c(0.01,0.7,0.01,0.7)) + p2 %>% pick_by_tree(t2) + plot_layout(widths = c(2,2))
ggsave(file=outfile_svg, plot=fig, device=svglite, width=2800, height=1800, units="px")
ggsave(plot=fig, width=1600, height=1200, units="px", dpi=300, filename=outfile_pdf, useDingbats=FALSE)
fig